In [14]:
import numpy as np
from sklearn.decomposition import PCA

import sys
sys.path.append('../general')
from pool import Pool
from utils import predict_positions
from metric import metric

import xgboost as xgb

In [2]:
pool = Pool('../data')

In [4]:
pca = PCA(500)

In [5]:
pca.fit(pool.features)

PCA(copy=True, iterated_power='auto', n_components=500, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [9]:
new_features = pca.transform(pool.features)

In [10]:
old_features = pool.features
pool.features = new_features

In [12]:
np.shape(pool.features), np.shape(old_features)

((90000, 500), (90000, 1052))

In [20]:
positions_one_hot = np.array([
    [0 if position != current_position else 1 for position in pool.POSITIONS]
    for current_position in pool.positions
])
pool.features_with_positions = np.concatenate((pool.features, positions_one_hot), axis=1)

In [21]:
def test_regression(model, pool, trys=10):
    scores = []
    for i in range(trys):
        train_pool, test_pool = pool.train_test_split()
        model.fit(train_pool.features_with_positions, train_pool.targets)
        positions = predict_positions(test_pool.features, model)
        scores.append(metric(positions, test_pool.positions, test_pool.targets, test_pool.probas))
    return scores

In [22]:
scores = test_regression(xgb.XGBRegressor(), pool)

In [23]:
np.mean(scores)

-0.005419755189839759

In [24]:
np.std(scores)

0.006192041998093268

In [25]:
scores

[-0.009759279571850554,
 0.0020983619771843065,
 -0.0114563559368969,
 -0.004999340311325385,
 0.002287680529473698,
 -0.015381700301122984,
 0.0014189943860032615,
 -0.002225021942810836,
 -0.0035320420160309452,
 -0.012648848711021262]